# Multiples Tools Usage

In [ ]:

import os

from llama_stack_client import Agent
from llama_stack.distribution.library_client import LlamaStackAsLibraryClient
from llama_stack_client.types import Document

model = "meta-llama/Llama-3.2-3B-Instruct"
os.environ["INFERENCE_MODEL"] = model
client = LlamaStackAsLibraryClient('ollama')
client.initialize()

agent = Agent(
  client,
  model=model,
  instructions="You are an helpful agent",
  tools=["builtin::rag/knowledge_search", "builtin::code_interpreter"],
)

documents = [
    Document(
        document_id="code-1",
        content="https://raw.githubusercontent.com/JuanmaBM/llama-stack-basics/refs/heads/main/operations.py",
        mime_type="text/plain",
        metadata={},
    )
]

response = agent.create_turn(
  messages=[{"role": "user", "content": "Fix the code, execute and print the output"}],
  stream=False,
  session_id=agent.create_session('new_session'),
  documents=documents
)

print(response.output_message.content)

# RAG with PostgreSQL as VectorDB

In [ ]:
import os
from llama_stack_client import Agent, AgentEventLogger
from llama_stack.distribution.library_client import LlamaStackAsLibraryClient
from llama_stack_client.types import Document
import uuid

model = "meta-llama/Llama-3.2-3B-Instruct"
os.environ["INFERENCE_MODEL"] = model
client = LlamaStackAsLibraryClient('./ollama-postgresql.yaml')
client.initialize()

# Create a vector database instance
embed_lm = next(m for m in client.models.list() if m.model_type == "embedding")
embedding_model = embed_lm.identifier
vector_db_id = f"v{uuid.uuid4().hex}"
client.vector_dbs.register(
    vector_db_id=vector_db_id,
    provider_id="pgvector",
    embedding_model=embedding_model,
)

# Create Documents
urls = [
    "memory_optimizations.rst",
    "chat.rst",
    "llama3.rst",
    "qat_finetune.rst",
    "lora_finetune.rst",
]
documents = [
    Document(
        document_id=f"num-{i}",
        content=f"https://raw.githubusercontent.com/pytorch/torchtune/main/docs/source/tutorials/{url}",
        mime_type="text/plain",
        metadata={},
    )
    for i, url in enumerate(urls)
]

# Insert documents
client.tool_runtime.rag_tool.insert(
    documents=documents,
    vector_db_id=vector_db_id,
    chunk_size_in_tokens=512,
)

# Get the model being served
llm = next(m for m in client.models.list() if m.model_type == "llm")
model = llm.identifier

# Create the RAG agent
rag_agent = Agent(
    client,
    model=model,
    instructions="You are a helpful assistant. Use the RAG tool to answer questions as needed.",
    tools=[
        {
            "name": "builtin::rag/knowledge_search",
            "args": {"vector_db_ids": [vector_db_id]},
        }
    ],
)

session_id = rag_agent.create_session(session_name=f"s{uuid.uuid4().hex}")

turns = ["what is torchtune", "tell me about dora"]

for t in turns:
    print("user>", t)
    stream = rag_agent.create_turn(
        messages=[{"role": "user", "content": t}], session_id=session_id, stream=True
    )
    for event in AgentEventLogger().log(stream):
        event.print()